In [130]:
import numpy as np
import pandas as pd

In [131]:
docs = read_n_documents(0,3000,'arxiv-metadata-oai-snapshot.json')
train_docs = process(docs)

docs = read_n_documents(3000,6000,'arxiv-metadata-oai-snapshot.json')
test_docs = process(docs)
test_docs

,title,version,abstract,authors_parsed,category_groups
3000,Multidimensional supernova simulations with ap...,2007,By 2D hydrodynamic simulations including a d...,"[[Scheck, L., , MPI for Astrophysics, Garching...",[astro-ph]
3001,On the inverse braid monoid,2007,Inverse braid monoid describes a structure o...,"[[Vershinin, Vladimir V., ]]","[math, math]"
3002,Deformation principle and further geometrizati...,2007,The space-time geometry is considered to be ...,"[[Rylov, Yuri A., ]]",[physics]
3003,Four-Quark Condensates in Nucleon QCD Sum Rules,2007,The in-medium behavior of the nucleon spectr...,"[[Thomas, R., ], [Hilger, T., ], [Kampfer, B., ]]","[hep-ph, nucl-th]"
3004,A balanced memory network,2007,A fundamental problem in neuroscience is und...,"[[Roudi, Yasser, ], [Latham, Peter E., ]]","[q-bio, cond-mat]"
...,...,...,...,...,...
5995,Network structure of chaotic patterns,2007,We reduce complex stripped patterns to a bas...,"[[Krishan, Kapilanjan, ]]","[nlin, nlin]"
5996,Testing the Accuracy of Synthetic Stellar Libr...,2007,In the present work we compare in detail som...,"[[Martins, Lucimara, ], [Coelho, Paula, ]]",[astro-ph]
5997,Minimal Pati-Salam Model from String Theory Un...,2007,We provide what we believe is the minimal th...,"[[Dent, James B., ], [Kephart, Thomas W., ]]",[hep-ph]
5998,"Critique of ""Quantum Enigma:Physic encounters ...",2007,The central claim that understanding quantum...,"[[Nauenberg, Michael, ]]",[quant-ph]


In [132]:
docs = read_n_documents(6000,9000,'arxiv-metadata-oai-snapshot.json')
val_docs = process(docs)

In [133]:
train_val = pd.concat([train_docs, val_docs])

words = processed_documents['abstract'].str.lower().str.split()
words

vocabulary = []
for i in range(6000):
    for j in words[i]:
        if j not in vocabulary:
            vocabulary.append(j)


import math

idf = [0] * len(vocabulary)

for k in range(len(vocabulary)):
    for i in range(6000):
        if vocabulary[k] in words[i]:
            idf[k] += 1

    idf[k] = math.log(6000/idf[k])
    print(k)
    
idf

In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfIdf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True, max_features=60000)

X_train = tfIdf.fit_transform(train_docs['abstract'])
X_test = tfIdf.fit_transform(test_docs['abstract'])

In [135]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=12, n_jobs=5)

In [136]:
from sklearn.model_selection import StratifiedKFold

stratKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)

In [137]:
from sklearn.model_selection import cross_val_score

cv_results = cross_val_score(log, X_train, val_docs["version"], cv=stratKFold, scoring='f1_micro')

c:\Users\Rudy\.conda\envs\nlp\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [138]:
cv_results, cv_results.mean()

(array([0.865     , 0.87      , 0.86833333, 0.86833333, 0.86833333]), 0.868)

In [139]:
log.fit(X_train, val_docs['version'])

LogisticRegression(C=50.0, multi_class='multinomial', n_jobs=5, random_state=12)

In [140]:
import eli5

eli5.show_weights(estimator=log, 
                  feature_names= list(tfIdf.get_feature_names_out()),
                 top=(5, 5))

In [141]:
prediction = log.predict(X_test)
result = pd.DataFrame(prediction, columns=['prediction'])
result["actual"] = test_docs["version"].reset_index(drop=True)
result

,prediction,actual
0,2007,2007
1,2007,2007
2,2007,2007
3,2007,2007
4,2007,2007
...,...,...
2995,2007,2007
2996,2007,2007
2997,2007,2007
2998,2007,2007


In [142]:
x = 0
for i in range(len(result["actual"])):
    if result["actual"][i] == result["prediction"][i]:
        x += 1

accuracy = x/len(result["actual"])
accuracy

0.8743333333333333